In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import itertools

########################
# Data

def load_csv(file_path):
    # CSV 파일 로드
    data = np.loadtxt(file_path, delimiter=',', skiprows=1)
    
    # 입력 변수와 목표 변수 분리 ('fetal_health'를 목표 변수로 설정)
    x_data = data[:, :-1]  # 마지막 열 제외 (입력 변수)
    y_data = data[:, -1].astype(int)  # 마지막 열 (목표 변수)
    
    # 목표 변수를 one-hot 인코딩으로 변환
    y_data = tf.one_hot(y_data - 1, depth=3).eval(session=tf.Session())

    return x_data, y_data

def split_data(x_data, y_data):
    # 데이터셋을 학습, 검증, 테스트로 나누기
    # random_state는 Seed값으로 동일한 분할 결과를 얻기 위해 사용
    # stratify는 클래스 비율 유지를 위해 사용
    # 전체 데이터의 20%를 테스트 데이터로 사용
    x_temp, x_test, y_temp, y_test = train_test_split(
        x_data, y_data, test_size=0.2, random_state=42, stratify=y_data)
    # 테스트 데이터를 제외한 나머지 데이터의 12.5%를 검증 데이터로 사용, 나머지를 학습 데이터로 사용
    x_train, x_val, y_train, y_val = train_test_split(
        x_temp, y_temp, test_size=0.125, random_state=42, stratify=y_temp)

    # 표준화 (스케일링)
    sc = StandardScaler()
    # 학습 데이터에 fit_transform을 사용하여 평균과 표준편차를 계산하고 데이터를 표준화
    x_train = sc.fit_transform(x_train)
    # 검증 및 테스트 데이터에는 학습 데이터의 평균과 표준편차를 사용하여 동일하게 표준화
    x_test = sc.transform(x_test)
    x_val = sc.transform(x_val)
    
    return x_train, x_val, x_test, y_train, y_val, y_test

########################
# Learning

def create_model(optimizer, input_num, hidden_sizes):
    tf.reset_default_graph()  # 그래프 초기화
    x = tf.placeholder(tf.float32, [None, input_num], name='x')  # 입력 데이터 플레이스홀더
    y = tf.placeholder(tf.float32, [None, 3], name='y')  # 레이블 데이터 플레이스홀더 (3개의 클래스)

    # 히든 레이어 생성
    layer_input = x
    layer_input_size = input_num
    for idx, hidden_size in enumerate(hidden_sizes):
        # 가중치 및 바이어스 초기화
        # tf.random.normal - 정규 분포를 따르는 랜덤 값 생성, 가중치 초기화용
        W_hidden = tf.Variable(tf.random.normal([layer_input_size, hidden_size], stddev=0.1), name=f'W_hidden_{idx+1}')
        b_hidden = tf.Variable(tf.zeros([hidden_size]), name=f'b_hidden_{idx+1}')
        # 레이어 출력 계산
        layer_output = tf.nn.relu(tf.matmul(layer_input, W_hidden) + b_hidden, name=f'relu_{idx+1}')
        # 다음 레이어를 위해 업데이트
        layer_input = layer_output
        layer_input_size = hidden_size

    # 출력 레이어 (3개의 클래스에 대한 확률)
    W_output = tf.Variable(tf.random.normal([layer_input_size, 3], stddev=0.1), name='W_output')
    b_output = tf.Variable(tf.zeros([3]), name='b_output')
    logits = tf.matmul(layer_input, W_output) + b_output
    prediction = tf.nn.softmax(logits, name='prediction')

    # 손실 함수 (크로스 엔트로피) 및 최적화 방법 정의
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits), name='loss')
    train_step = optimizer.minimize(loss, name='train_step')

    # 정확도 계산
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

    return x, y, prediction, loss, train_step, accuracy

# 모델 학습 및 평가 함수
def execute_model(x, y, prediction, loss, train_step, accuracy, data, epochs, batch_size):
    train_losses = []
    val_losses = []
    test_losses = []
    train_accuracies = []
    val_accuracies = []
    test_accuracies = []

    x_train, x_val, x_test, y_train, y_val, y_test = data
       
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())  # 변수 초기화
        for epoch in range(1, epochs + 1):
            # 학습 데이터 셔플
            permutation = np.random.permutation(len(x_train))
            x_train_shuffled = x_train[permutation]
            y_train_shuffled = y_train[permutation]

            num_batches = (len(x_train) + batch_size - 1) // batch_size
            for i in range(num_batches):
                start_idx = i * batch_size
                end_idx = min((i+1) * batch_size, len(x_train))
                batch_x = x_train_shuffled[start_idx:end_idx]
                batch_y = y_train_shuffled[start_idx:end_idx]
                sess.run(train_step, feed_dict={x: batch_x, y: batch_y})

            if epoch % 10 == 0 or epoch == 1:
                # 손실 및 정확도 계산
                train_loss, train_acc = sess.run([loss, accuracy], feed_dict={x: x_train, y: y_train})
                val_loss, val_acc = sess.run([loss, accuracy], feed_dict={x: x_val, y: y_val})
                test_loss, test_acc = sess.run([loss, accuracy], feed_dict={x: x_test, y: y_test})
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                test_losses.append(test_loss)
                train_accuracies.append(train_acc)
                val_accuracies.append(val_acc)
                test_accuracies.append(test_acc)
                print(f'Epoch {epoch}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

        # 최종 예측 수행
        predictions = sess.run(prediction, feed_dict={x: x_test})

    return train_losses, val_losses, test_losses, train_accuracies, val_accuracies, test_accuracies, predictions

# 모델 학습 및 결과 도출
def learn(data, param_grid):
    print('1. Training Models')

    x_train = data[0]
    best_report = None
    best_test_accuracy = -1

    # 하이퍼파라미터 조합 생성
    keys = param_grid.keys()
    combinations = list(itertools.product(*(param_grid[key] for key in keys)))

    print(f"Total parameter combinations: {len(combinations)}")

    for idx, combo in enumerate(combinations, 1):
        params = dict(zip(keys, combo))
        print(f"Training model {idx}/{len(combinations)} with parameters:")
        print(f"Hidden sizes: {params['hidden_sizes']}, Optimizer: {params['optimizer'].__name__}, Learning rate: {params['learning_rate']}, Batch size: {params['batch_size']}, Epochs: {params['epochs']}")

        optimizer_class = params['optimizer']
        learning_rate = params['learning_rate']
        hidden_sizes = params['hidden_sizes']
        epochs = params['epochs']
        batch_size = params['batch_size']

        # 옵티마이저 인스턴스 생성
        optimizer = optimizer_class(learning_rate=learning_rate)

        # 모델 생성
        x, y, prediction, loss, train_step, accuracy = create_model(optimizer, x_train.shape[1], hidden_sizes)

        # 모델 실행 및 결과 수집
        train_losses, val_losses, test_losses, train_accuracies, val_accuracies, test_accuracies, predictions = execute_model(
            x, y, prediction, loss, train_step, accuracy, data, epochs, batch_size
        )

        # 결과 출력
        print(f"Optimizer: {optimizer_class.__name__}, Learning rate: {learning_rate}")
        print(f"Hidden sizes: {hidden_sizes}, Batch size: {batch_size}, Epochs: {epochs}")
        print(f"Training loss: {train_losses[-1]:.4f}")
        print(f"Validation loss: {val_losses[-1]:.4f}")
        print(f"Test loss: {test_losses[-1]:.4f}")
        print(f"Test Accuracy: {test_accuracies[-1]:.4f}\n")

        # 최고 성능 모델 업데이트 (테스트 정확도가 가장 높은 모델 선택)
        if test_accuracies[-1] > best_test_accuracy:
            best_test_accuracy = test_accuracies[-1]
            best_report = {
                "params": params,
                "train_losses": train_losses,
                "val_losses": val_losses,
                "test_losses": test_losses,
                "train_accuracies": train_accuracies,
                "val_accuracies": val_accuracies,
                "test_accuracies": test_accuracies,
                "predictions": predictions,
                "y_test": data[-1]
            }
            print(f"New best model found: Test Accuracy = {best_test_accuracy:.4f}\n")

        print("-" * 80)

    return best_report

########################
# Report

def generate_report(report):
    print('2. Conclusion')
    if report is None:
        print("No report to generate.")
        return
    params = report['params']
    print(f"Best Parameters:")
    print(f"Optimizer: {params['optimizer'].__name__}")
    print(f"Learning rate: {params['learning_rate']}")
    print(f"Hidden sizes: {params['hidden_sizes']}")
    print(f"Batch size: {params['batch_size']}")
    print(f"Epochs: {params['epochs']}")
    print(f"Training loss: {report['train_losses'][-1]:.4f}")
    print(f"Validation loss: {report['val_losses'][-1]:.4f}")
    print(f"Test loss: {report['test_losses'][-1]:.4f}")
    print(f"Test Accuracy: {report['test_accuracies'][-1]:.4f}")

    # 실제 값과 예측 값 출력
    y_test = report['y_test']
    predictions = report['predictions']
    print("\nActual Values vs Predicted Values:")
    for actual, predicted in zip(y_test.argmax(axis=1), predictions.argmax(axis=1)):
        print(f"Actual: {actual}, Predicted: {predicted}")

########################
# 파라미터 그리드 설정

# 3(hidden_sizes) * 3(optimizer) * 3(learning_rate) * 3(batch_size) * 3(epochs) = 243 개의 모델을 생성
param_grid = {
    # 뉴런 수 (Ex. [32, 16, 8]로 설정하면 3개의 히든 레이어가 생성 되고 뉴런 수는 각각 32, 16, 8로 할당)
    'hidden_sizes': [[32, 16, 8], [16, 16, 16], [8, 16, 32]],
    # 옵티마이저 -
    # tf.train.GradientDescentOptimizer, tf.train.AdamOptimizer, tf.train.AdagradOptimizer
    'optimizer': [tf.train.GradientDescentOptimizer, tf.train.AdamOptimizer, tf.train.AdagradOptimizer],
    # 학습률
    'learning_rate': [0.01, 0.001, 0.0001],
    # 배치 사이즈
    'batch_size': [16, 32, 64],
    # Epochs
    'epochs': [100, 200, 300]
}

########################
# 메인 실행 흐름

def main():
    # 데이터 로드 및 전처리
    x_data, y_data = load_csv('fetal_health.csv')
    x_train, x_val, x_test, y_train, y_val, y_test = split_data(x_data, y_data)
    data = (x_train, x_val, x_test, y_train, y_val, y_test)

    # Learning
    best_report = learn(data, param_grid)

    # Report
    generate_report(best_report)

if __name__ == "__main__":
    main()


1. Training Models
Total parameter combinations: 243
Training model 1/243 with parameters:
Hidden sizes: [32, 16, 8], Optimizer: GradientDescentOptimizer, Learning rate: 0.01, Batch size: 16, Epochs: 100
Epoch 1, Training Loss: 0.8863, Training Accuracy: 0.7781, Validation Loss: 0.8849, Validation Accuracy: 0.7793, Test Loss: 0.8852, Test Accuracy: 0.7793
Epoch 10, Training Loss: 0.6535, Training Accuracy: 0.7781, Validation Loss: 0.6504, Validation Accuracy: 0.7793, Test Loss: 0.6507, Test Accuracy: 0.7793
Epoch 20, Training Loss: 0.3544, Training Accuracy: 0.8507, Validation Loss: 0.3600, Validation Accuracy: 0.8545, Test Loss: 0.3679, Test Accuracy: 0.8451
Epoch 30, Training Loss: 0.2568, Training Accuracy: 0.8944, Validation Loss: 0.2804, Validation Accuracy: 0.8967, Test Loss: 0.2892, Test Accuracy: 0.8920
Epoch 40, Training Loss: 0.2244, Training Accuracy: 0.9059, Validation Loss: 0.2537, Validation Accuracy: 0.8920, Test Loss: 0.2630, Test Accuracy: 0.8897
Epoch 50, Training Los